In [46]:
import os
from os.path import join as p_join
import sys
sys.path.insert(0, '..')
import time
from tqdm.notebook import tqdm
import itertools
import collections
from datetime import datetime
from dateutil.relativedelta import relativedelta

import numpy as np
import pandas as pd
import scipy
from matplotlib_venn import *
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import requests
pd.set_option('max.columns', 100)

In [9]:
DATE = '2020-01-01'
PROJECT_PATH = '..'

In [137]:
from src.parse_utils import (
    parse_supplies,
    parse_purchases,
    parse_sales,
    parse_storage,
    parse_report
)

In [4]:
matplotlib.rcParams['figure.figsize'] = (8, 8)
sns.set_style('whitegrid')

---

In [68]:
wb_key = open(p_join(PROJECT_PATH, 'configs', 'wildberries_api64.txt'), mode='r', encoding='utf-8').read()
var_name_dict = json.load(
    open(p_join(PROJECT_PATH, 'configs', 'wb_columns_dict.json'), mode='r', encoding='utf-8')
)

In [155]:
# Поставки
supplies_df = parse_supplies(
    date_from='2021-12-31',
    wb_key=wb_key,
#     var_name_dict=var_name_dict
)
supplies_df

""


In [138]:
# Склад
storage_df = parse_storage(
    date_from='2021-12-31',
    wb_key=wb_key,
#     var_name_dict=var_name_dict
)
storage_df

In [140]:
# Заказы
purchase_df = parse_purchases(
    date_from='2021-12-31',
    wb_key=wb_key,
    flag=0,
#     var_name_dict=var_name_dict
)
purchase_df

,number,date,lastChangeDate,supplierArticle,techSize,barcode,quantity,totalPrice,discountPercent,warehouseName,oblast,incomeID,odid,nmId,subject,category,brand,isCancel,cancel_dt,gNumber
0,42205820799,2021-12-30 23:40:14,2021-12-31 01:28:09,mskmsk,0,2002387186009,1,780,50,Коледино,Рязанская,5560043,200536645350,26999508,Маски для сна,Аксессуары,syho,False,0001-01-01T00:00:00,3820670233297566848
1,42198293887,2021-12-30 21:00:32,2021-12-31 03:12:28,teasp_вет/розовое_золото,0,2002727191007,1,930,50,Коледино,,0,150464249341,28222072,Ложки чайные,Посуда и инвентарь,Auplum,False,0001-01-01T00:00:00,98681236305412013130
2,42198293887,2021-12-30 21:00:32,2021-12-31 03:12:28,teasp_вет/розовое_золото,0,2002727191007,1,930,50,Коледино,,0,150464249342,28222072,Ложки чайные,Посуда и инвентарь,Auplum,False,0001-01-01T00:00:00,98681236305412013130


In [142]:
# Продажи
sales_df = parse_sales(
    date_from='2021-12-31',
    wb_key=wb_key,
    flag=0,
#     var_name_dict=var_name_dict
)
sales_df

,number,date,lastChangeDate,supplierArticle,techSize,barcode,quantity,totalPrice,discountPercent,isSupply,isRealization,orderId,promoCodeDiscount,warehouseName,countryName,oblastOkrugName,regionName,incomeID,saleID,odid,spp,forPay,finishedPrice,priceWithDisc,nmId,subject,category,brand,IsStorno,gNumber
0,,2021-12-31 10:12:21,2021-12-31 10:16:10,org01pink,0,2007029886002,1,2534,50,False,True,40988540319,25,Коледино,Россия,Уральский федеральный округ,Ямало-Ненецкий,3964378,S2209830180,150153581792,0,836.22,950,950.25,38290083,Органайзеры для хранения,Хранение вещей,Auplum,0,97171068044453121889
1,,2021-12-31 11:23:26,2021-12-31 11:26:06,НСП/006_черный_глянцевый,0,2001256882004,1,4240,57,False,True,41866317631,25,Коледино,Россия,Южный федеральный округ,Аксайский,4804244,S2210380660,200408212099,0,1203.31,1367,1367.40,19464862,Наборы столовых приборов,Посуда и инвентарь,Auplum,0,5197214683302349386
2,,2021-12-31 11:44:11,2021-12-31 11:47:06,mskmsk,0,2002387186009,1,1040,50,False,True,41966154335,25,Коледино,Россия,Южный федеральный округ,Краснодарский,3964378,S2210585930,150380390304,0,331.50,390,390.00,26999508,Маски для сна,Аксессуары,syho,0,9687123685492583728
3,,2021-12-31 11:48:25,2021-12-31 11:54:08,petklt01black,0,2006959886007,1,1331,50,False,True,42125120383,25,Коледино,Россия,Центральный федеральный округ,Москва,3964378,S2210621846,200499123974,0,439.23,499,499.13,38129104,Колтунорезки,Товары для животных,imilive,0,2995193404110514876
4,,2021-12-31 11:48:25,2021-12-31 11:54:08,mskmsk,0,2002387186009,1,1040,50,False,True,42125120383,25,Коледино,Россия,Центральный федеральный округ,Москва,5560043,S2210626505,200499123973,0,331.50,390,390.00,26999508,Маски для сна,Аксессуары,syho,0,2995193404110514876
5,,2021-12-31 12:00:28,2021-12-31 12:08:14,bandage1,L,2002852068014,1,2264,50,False,True,42070596383,25,Казань,Россия,Приволжский федеральный округ,Пермский,3964378,S2210765643,200476679475,0,721.65,849,849.00,28678954,Бандажи коленные,Здоровье,syho,0,1502218213606833431
6,,2021-12-31 13:56:30,2021-12-31 14:00:40,bandage1,XL,2002852035009,1,2264,57,False,True,42109495039,25,Коледино,Россия,Центральный федеральный округ,Москва,3964378,S2211910326,200494376624,0,657.13,730,730.14,28678954,Бандажи коленные,Здоровье,syho,0,93192745789444551391
7,1493353733,2021-12-31 15:54:17,2021-12-31 15:59:21,НСП/007_серебряный_глянцевый,0,2001256892003,-1,-4240,59,False,True,41304069919,25,Коледино,Россия,Южный федеральный округ,Краснодарский,4804244,R88864188,150219300598,0,-1147.34,-1303,-1303.80,19464863,Наборы столовых приборов,Посуда и инвентарь,Auplum,0,7725577098191793901
8,1493354392,2021-12-31 15:54:29,2021-12-31 15:59:21,НСП/007_серебряный_глянцевый,0,2001256892003,-1,-4240,59,False,True,41304069919,25,Коледино,Россия,Южный федеральный округ,Краснодарский,4804244,R88864242,150219300597,0,-1147.34,-1303,-1303.80,19464863,Наборы столовых приборов,Посуда и инвентарь,Auplum,0,7725577098191793901
9,1493354794,2021-12-31 15:54:37,2021-12-31 15:59:21,НСП/007_серебряный_глянцевый,0,2001256892003,-1,-4240,0,True,False,41304069919,0,Коледино,Россия,Южный федеральный округ,Краснодарский,0,R88864288,150219300595,0,0.00,0,0.00,19464863,Наборы столовых приборов,Посуда и инвентарь,Auplum,0,7725577098191793901


In [143]:
# Отчёт о продажах по реализации
report_df = parse_report(
    date_from='2021-12-31',
    wb_key=wb_key,
    limit=100_000,
#     var_name_dict=var_name_dict
)
report_df

""


In [53]:
(
    set(supplies_df.columns) 
    & set(storage_df.columns)
    & set(purchase_df.columns)
    & set(sales_df.columns)
#     & set(report_df.columns)
)

{'barcode',
 'lastChangeDate',
 'nmId',
 'quantity',
 'supplierArticle',
 'techSize',
 'warehouseName'}

In [74]:
import sqlalchemy as sa

In [231]:
sql_db_credentials = json.load(
    open(p_join(PROJECT_PATH, 'configs', 'sql_db_creadentials.json'), mode='r', encoding='utf-8')
)

In [232]:
db = 'yarik'
eng = sa.create_engine(
    f"mysql+pymysql://{sql_db_credentials[db]['login']}:{sql_db_credentials[db]['password']}@{sql_db_credentials[db]['host']}:{sql_db_credentials[db]['port']}"
)
eng.connect()

In [158]:
pd.read_sql(
    """
    show databases
    """,
    eng
)

,Database
0,information_schema
1,mp_db
2,mysql
3,performance_schema
4,sys
5,wb_yarik


In [89]:
sales_df.to_sql(
    schema='wb_yarik',
    name='sales',
    con=eng,
    if_exists='append'
)

In [90]:
purchase_df.to_sql(
    schema='wb_yarik',
    name='purchases',
    con=eng,
    if_exists='append'
)

In [91]:
storage_df.to_sql(
    schema='wb_yarik',
    name='storage',
    con=eng,
    if_exists='append'
)

In [92]:
supplies_df.to_sql(
    schema='wb_yarik',
    name='supplies',
    con=eng,
    if_exists='append'
)

In [93]:
report_df.to_sql(
    schema='wb_yarik',
    name='report',
    con=eng,
    if_exists='append'
)

In [249]:
tmp_df = pd.read_sql(
    """
    select 
        *
    from
        wb_yarik.sales
    """,
    eng
)
tmp_df

,index,number,date,lastChangeDate,supplierArticle,techSize,barcode,quantity,totalPrice,discountPercent,isSupply,isRealization,orderId,promoCodeDiscount,warehouseName,countryName,oblastOkrugName,regionName,incomeID,saleID,odid,spp,forPay,finishedPrice,priceWithDisc,nmId,subject,category,brand,IsStorno,gNumber
0,0,,2021-09-01 10:19:45,2021-09-01 10:30:51,НСП/004_золотой_глянцевый,0,2001256726001,1,2900,30,0,1,3.054720e+10,25,Новосибирск,Россия,Сибирский федеральный округ,Новосибирская,3537166,S1741183025,100649433498,0,1385.47,1386.47,1522.50,19464860,Наборы столовых приборов,Посуда и инвентарь,Auplum,0,91042353372731518546
1,1,,2021-09-01 14:06:51,2021-09-01 14:14:54,КН/002_мятный,0,2001254428006,1,4104,30,0,1,3.055047e+10,25,Коледино,Россия,Центральный федеральный округ,Москва,2267245,S1742126327,100649515477,0,1960.69,1961.69,2154.60,19463070,Наборы кухонных принадлежностей,Посуда и инвентарь,Auplum,0,9859120058773614172
2,2,1272674075,2021-09-01 15:45:31,2021-09-02 02:23:36,КН/002_мятный,0,2001254428006,-1,-4104,30,0,1,3.055047e+10,25,Коледино,Россия,Центральный федеральный округ,Москва,2267245,R68974662,100649515477,0,-1960.69,-1961.69,-2154.60,19463070,Наборы кухонных принадлежностей,Посуда и инвентарь,Auplum,0,9859120058773614172
3,3,,2021-09-02 11:29:10,2021-09-02 11:33:36,НСП/011_серебряный_матовый,0,2001256932006,1,2695,30,0,1,3.069453e+10,25,Екатеринбург,Россия,Уральский федеральный округ,Тюменская,3537166,S1744466980,100653555980,0,1287.54,1288.54,1414.88,19464867,Наборы столовых приборов,Посуда и инвентарь,Auplum,0,91195127552123899025
4,4,,2021-09-02 14:44:08,2021-09-02 14:49:39,mskmsk,0,2002387186009,1,1332,30,0,1,3.065170e+10,25,Санкт-Петербург,Россия,Северо-западный федеральный округ,Санкт-Петербург,3247649,S1745288671,100652352563,0,594.40,605.00,699.30,26999508,Маски для сна,Аксессуары,syho,0,96786532666405045542
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1710,1710,,2021-12-30 18:00:37,2021-12-30 18:05:15,org01pink,0,2007029886002,1,2534,65,0,1,4.209472e+10,25,Коледино,Россия,Центральный федеральный округ,Москва,3964378,S2207926643,200489509254,0,618.62,665.00,665.18,38290083,Органайзеры для хранения,Хранение вещей,Auplum,0,5959688921835646454
1711,1711,1491020269,2021-12-30 18:25:19,2021-12-30 18:33:13,mskmsk,0,2002387186009,-1,-1040,50,0,1,4.122078e+10,25,Коледино,Россия,Приволжский федеральный округ,Нижегородская,3964378,R88691368,150205302930,0,-331.50,-390.00,-390.00,26999508,Маски для сна,Аксессуары,syho,0,8381100090786046259
1712,1712,,2021-12-30 19:44:17,2021-12-30 19:50:25,mskmsk,0,2002387186009,1,1040,50,0,1,4.183727e+10,25,Коледино,Россия,Центральный федеральный округ,Раменский,3964378,S2208849791,200402127574,0,331.50,390.00,390.00,26999508,Маски для сна,Аксессуары,syho,0,7186667665181278894
1713,1713,,2021-12-30 20:03:41,2021-12-30 20:11:06,bandage1,XL,2002852035009,1,2264,50,0,1,4.206951e+10,25,Коледино,Россия,Центральный федеральный округ,Люберецкий,3964378,S2208987276,200476393733,0,721.65,849.00,849.00,28678954,Бандажи коленные,Здоровье,syho,0,7517348191030456791
